In [2]:
import os
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
from ast import literal_eval
import random
imp
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', 500)

In [4]:
def flatten_df(csv_path='Data/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'},
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = pd.io.json.json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    
    df['hits'][df['hits'] == "[]"] = "[{}]"
    df['hits'] = df['hits'].apply(literal_eval).str[0]
    hits = pd.io.json.json_normalize(df['hits'])
    hits = hits.drop(['product', 'promotion'], axis=1)
    
    df = pd.concat([df, hits], axis=1, sort=False)
    df = df.drop(['customDimensions', 'hits', 'customMetrics', 'customVariables', 'experiment', 'publisher_infos'], axis=1)
    
    print(f"Shape: {df.shape}")
    return df

In [4]:
%%time

train = flatten_df(nrows = 1000)

C:\Users\samarth\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Shape: (1000, 105)
Wall time: 6.77 s


In [5]:
train.head()

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,appInfo.exitScreenName,appInfo.landingScreenName,appInfo.screenDepth,appInfo.screenName,contentGroup.contentGroup1,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.contentGroup4,contentGroup.contentGroup5,contentGroup.contentGroupUniqueViews1,contentGroup.contentGroupUniqueViews2,contentGroup.contentGroupUniqueViews3,contentGroup.previousContentGroup1,contentGroup.previousContentGroup2,contentGroup.previousContentGroup3,contentGroup.previousContentGroup4,contentGroup.previousContentGroup5,dataSource,eCommerceAction.action_type,eCommerceAction.step,eventInfo.eventAction,eventInfo.eventCategory,eventInfo.eventLabel,exceptionInfo.isFatal,hitNumber,hour,isEntrance,isExit,isInteraction,item.currencyCode,minute,page.hostname,page.pagePath,page.pagePathLevel1,page.pagePathLevel2,page.pagePathLevel3,page.pagePathLevel4,page.pageTitle,page.searchCategory,page.searchKeyword,promotionActionInfo.promoIsView,referer,social.hasSocialSourceReferral,social.socialInteractionNetworkAction,social.socialNetwork,time,transaction.currencyCode,type
0,Organic Search,20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,Western Europe,1,1,1,1,1,NaN,NaN,NaN,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,water bottle,organic,NaN,google,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/google+redesig...,0,shop.googlemerchandisestore.com/google+redesig...,(not set),Bags,(not set),(not set),(not set),NaN,1,NaN,(entrance),(entrance),(entrance),(entrance),(entrance),web,0,1,NaN,NaN,NaN,True,1,17,True,True,True,USD,0,shop.googlemerchandisestore.com,/google+redesign/bags/water+bottles+and+tumblers,/google+redesign/,/bags/,/water+bottles+and+tumblers,,Water Bottles & Tumblers | Drinkware | Google ...,NaN,NaN,NaN,https://www.google.co.uk/search?q=water+bottle...,No,:,(not set),0,USD,PAGE
1,Referral,20171016,8934116514970143966,Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not availabl

In [6]:
def clean_df(df):
    df['totals.transactionRevenue'] = df['totals.transactionRevenue'].fillna(0)
    df['totals.totalTransactionRevenue'] = df['totals.totalTransactionRevenue'].fillna(0)
    df['totals.transactions'] = df['totals.transactions'].fillna(0)
    
    keep_cols = ['channelGrouping','fullVisitorId', 'date', 'visitNumber', 'device.browser',
                'device.deviceCategory', 'device.operatingSystem', 'geoNetwork.continent', 'geoNetwork.country', 
                'geoNetwork.region', 'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews', 
                'totals.sessionQualityDim', 'totals.timeOnSite', 'totals.totalTransactionRevenue', 
                'totals.transactionRevenue', 'totals.transactions', 'trafficSource.adContent',
                'trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.page', 
                'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign', 'trafficSource.keyword', 
                'trafficSource.source', 'appInfo.exitScreenName', 'contentGroup.contentGroup2', 
                'contentGroup.contentGroup3', 'eventInfo.eventAction']
    df = df[keep_cols]
    
    df = df.replace(['not available in demo dataset', '(not provided)', '(not set)', ''], np.NaN)
    
    return df

In [7]:
%%time

train = clean_df(train)

Wall time: 45.9 ms


In [8]:
print(train.shape)
train.head()

(1000, 30)


,channelGrouping,fullVisitorId,date,visitNumber,device.browser,device.deviceCategory,device.operatingSystem,geoNetwork.continent,geoNetwork.country,geoNetwork.region,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.source,appInfo.exitScreenName,contentGroup.contentGroup2,contentGroup.contentGroup3,eventInfo.eventAction
0,Organic Search,3162355547410993243,20171016,1,Firefox,desktop,Windows,Europe,Germany,NaN,1,1,1,1,1,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,water bottle,google,shop.googlemerchandisestore.com/google+redesig...,Bags,NaN,NaN
1,Referral,8934116514970143966,20171016,6,Chrome,desktop,Chrome OS,Americas,United States,California,NaN,2,NaN,2,2,28,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,sites.google.com,shop.googlemerchandisestore.com/asearch.html,NaN,NaN,NaN
2,Direct,7992466427990357681,20171016,1,Chrome,mobile,Android,Americas,United States,NaN,NaN,2,1,2,1,38,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,(direct),shop.googlemerchandisestore.com/google+redesig...,NaN,NaN,NaN
3,Organic Search,9075655783635761930,20171016,1,Chrome,desktop,Windows,Asia,Turkey,NaN,NaN,2,1,2,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,google,shop.googlemerchandisestore.com/google+redesig...,NaN,NaN,NaN
4,Organic Search,6960673291025684308,20171016,1,Chrome,desktop,Windows,Americas,Mexico,NaN,NaN,2,1,2,1,52,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,google,shop.googlemerchandisestore.com/google+redesig...,NaN,NaN,NaN


In [9]:
# print(train_small['channelGrouping'].unique())
# train_small['channelGrouping'] = train_small['channelGrouping'].astype('category')



In [10]:
def feature_date(df):
    year = []
    month = []
    day = []
    
    for each in df['date']:
        each = str(each)
        year.append(int(each[0:4]))
        month.append(int(each[4:6]))
        day.append(int(each[6:8]))
    
    Year = pd.DataFrame(year).rename(columns = {0:'Year'})
    Month = pd.DataFrame(month).rename(columns = {0:'Month'})
    Day = pd.DataFrame(day).rename(columns = {0:'Day'})
    
    df['date'] = df['date'].astype('str')
    df['date'] = pd.to_datetime(df['date'], yearfirst=True)
    
    y = df.index[((df['date'] == '2018-1-1') |
                ((df['date'] > '2018-2-12') & (df['date'] <= '2018-2-19')) |
                ((df['date'] > '2018-5-25') & (df['date'] <= '2018-5-28')) |
                ((df['date'] > '2018-6-29') & (df['date'] <= '2018-7-4')) |
                ((df['date'] > '2018-8-25') & (df['date'] <= '2018-8-31')) |
                ((df['date'] > '2018-9-1') & (df['date'] <= '2018-9-3')) |
                ((df['date'] > '2018-11-20') & (df['date'] <= '2018-11-28')) |
                ((df['date'] > '2018-12-20') & (df['date'] <= '2018-12-31')) |

                (df['date'] == '2017-1-1') |
                ((df['date'] > '2017-2-12') & (df['date'] <= '2017-2-19')) |
                ((df['date'] > '2017-5-26') & (df['date'] <= '2017-5-29')) |
                ((df['date'] > '2017-6-29') & (df['date'] <= '2017-7-4')) |
                ((df['date'] > '2017-8-25') & (df['date'] <= '2017-8-31')) |
                ((df['date'] > '2017-9-2') & (df['date'] <= '2017-9-4')) |
                ((df['date'] > '2017-11-21') & (df['date'] <= '2017-11-29')) |
                ((df['date'] > '2017-12-20') & (df['date'] <= '2017-12-31')) |

                (df['date'] == '2016-1-1') |
                ((df['date'] > '2016-2-12') & (df['date'] <= '2016-2-19')) |
                ((df['date'] > '2016-5-27') & (df['date'] <= '2016-5-30')) |
                ((df['date'] > '2016-6-29') & (df['date'] <= '2016-7-4')) |
                ((df['date'] > '2016-8-25') & (df['date'] <= '2016-8-31')) |
                ((df['date'] > '2016-9-2') & (df['date'] <= '2016-9-5')) |
                ((df['date'] > '2016-11-22') & (df['date'] <= '2016-11-30')) |
                ((df['date'] > '2016-12-20') & (df['date'] <= '2016-12-31')) |

                (df['date'] == '2019-1-1') |
                ((df['date'] > '2019-2-12') & (df['date'] <= '2016-2-19')))]
    
    is_holiday = []
    for index in range(0, df.shape[0]):
        if index in y:
            is_holiday.append(1)
        else:
            is_holiday.append(0)
    
    is_holiday = pd.DataFrame(is_holiday).rename(columns = {0:'is_holiday'})
    df = pd.concat([df, Year, Month, Day, is_holiday], axis = 1)
    df = df.drop(['date'], axis=1)
    
    return df

In [11]:
%%time

train = feature_date(train)

Wall time: 45.9 ms


In [12]:
print(train.shape)
train.head()

(1000, 33)


,channelGrouping,fullVisitorId,visitNumber,device.browser,device.deviceCategory,device.operatingSystem,geoNetwork.continent,geoNetwork.country,geoNetwork.region,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.source,appInfo.exitScreenName,contentGroup.contentGroup2,contentGroup.contentGroup3,eventInfo.eventAction,Year,Month,Day,is_holiday
0,Organic Search,3162355547410993243,1,Firefox,desktop,Windows,Europe,Germany,NaN,1,1,1,1,1,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,water bottle,google,shop.googlemerchandisestore.com/google+redesig...,Bags,NaN,NaN,2017,10,16,0
1,Referral,8934116514970143966,6,Chrome,desktop,Chrome OS,Americas,United States,California,NaN,2,NaN,2,2,28,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,sites.google.com,shop.googlemerchandisestore.com/asearch.html,NaN,NaN,NaN,2017,10,16,0
2,Direct,7992466427990357681,1,Chrome,mobile,Android,Americas,United States,NaN,NaN,2,1,2,1,38,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,(direct),shop.googlemerchandisestore.com/google+redesig...,NaN,NaN,NaN,2017,10,16,0
3,Organic Search,9075655783635761930,1,Chrome,desktop,Windows,Asia,Turkey,NaN,NaN,2,1,2,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,google,shop.googlemerchandisestore.com/google+redesig...,NaN,NaN,NaN,2017,10,16,0
4,Organic Search,6960673291025684308,1,Chrome,desktop,Windows,Americas,Mexico,NaN,NaN,2,1,2,1,52,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,google,shop.googlemerchandisestore.com/google+redesig...,NaN,NaN,NaN,2017,10,16,0


In [13]:
def feature_browser(df):
    browsers = ['Chrome', 'Safari', 'Samsung Internet', 'Firefox', 'Internet Explorer', 'Edge']
    device_browser = []

    for each in df['device.browser']:
        if each in browsers:
            device_browser.append(each)
        elif each == 'Safari (in-app)':
            device_browser.append('Safari')
        elif each == 'Mozilla':
            device_browser.append('Firefox')
        else:
            device_browser.append('Other')

    device_browser = pd.DataFrame(device_browser).rename(columns = {0:'device_browser'})      

    df = pd.concat([df, device_browser], axis = 1)
    df = df.drop(columns=['device.browser'])
    
    return df

In [14]:
%%time

train = feature_browser(train)

Wall time: 5.99 ms


In [15]:
print(train.shape)
train.head()

(1000, 33)


,channelGrouping,fullVisitorId,visitNumber,device.deviceCategory,device.operatingSystem,geoNetwork.continent,geoNetwork.country,geoNetwork.region,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.source,appInfo.exitScreenName,contentGroup.contentGroup2,contentGroup.contentGroup3,eventInfo.eventAction,Year,Month,Day,is_holiday,device_browser
0,Organic Search,3162355547410993243,1,desktop,Windows,Europe,Germany,NaN,1,1,1,1,1,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,water bottle,google,shop.googlemerchandisestore.com/google+redesig...,Bags,NaN,NaN,2017,10,16,0,Firefox
1,Referral,8934116514970143966,6,desktop,Chrome OS,Americas,United States,California,NaN,2,NaN,2,2,28,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,sites.google.com,shop.googlemerchandisestore.com/asearch.html,NaN,NaN,NaN,2017,10,16,0,Chrome
2,Direct,7992466427990357681,1,mobile,Android,Americas,United States,NaN,NaN,2,1,2,1,38,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,(direct),shop.googlemerchandisestore.com/google+redesig...,NaN,NaN,NaN,2017,10,16,0,Chrome
3,Organic Search,9075655783635761930,1,desktop,Windows,Asia,Turkey,NaN,NaN,2,1,2,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,google,shop.googlemerchandisestore.com/google+redesig...,NaN,NaN,NaN,2017,10,16,0,Chrome
4,Organic Search,6960673291025684308,1,desktop,Windows,Americas,Mexico,NaN,NaN,2,1,2,1,52,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,google,shop.googlemerchandisestore.com/google+redesig...,NaN,NaN,NaN,2017,10,16,0,Chrome


In [16]:
def feature_inbasket(df):
    basket = []
    for each in df['appInfo.exitScreenName']:
        each = str(each)
        if 'basket' in each:
            basket.append(1)
        else:
            basket.append(0)
    
    is_inbasket = pd.DataFrame(basket).rename(columns = {0:'is_inbasket'})
    
    df = pd.concat([df, is_inbasket], axis = 1)
    df = df.drop(columns=['appInfo.exitScreenName'])
    
    return df

In [17]:
%%time

train = feature_inbasket(train)

Wall time: 6.98 ms


In [18]:
print(train.shape)
train.head()

(1000, 33)


,channelGrouping,fullVisitorId,visitNumber,device.deviceCategory,device.operatingSystem,geoNetwork.continent,geoNetwork.country,geoNetwork.region,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.source,contentGroup.contentGroup2,contentGroup.contentGroup3,eventInfo.eventAction,Year,Month,Day,is_holiday,device_browser,is_inbasket
0,Organic Search,3162355547410993243,1,desktop,Windows,Europe,Germany,NaN,1,1,1,1,1,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,water bottle,google,Bags,NaN,NaN,2017,10,16,0,Firefox,0
1,Referral,8934116514970143966,6,desktop,Chrome OS,Americas,United States,California,NaN,2,NaN,2,2,28,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,sites.google.com,NaN,NaN,NaN,2017,10,16,0,Chrome,0
2,Direct,7992466427990357681,1,mobile,Android,Americas,United States,NaN,NaN,2,1,2,1,38,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,(direct),NaN,NaN,NaN,2017,10,16,0,Chrome,0
3,Organic Search,9075655783635761930,1,desktop,Windows,Asia,Turkey,NaN,NaN,2,1,2,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,google,NaN,NaN,NaN,2017,10,16,0,Chrome,0
4,Organic Search,6960673291025684308,1,desktop,Windows,Americas,Mexico,NaN,NaN,2,1,2,1,52,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,google,NaN,NaN,NaN,2017,10,16,0,Chrome,0


In [19]:
def feature_totals(df):
    df['totals.bounces'] = df['totals.bounces'].fillna(0)
    df['totals.newVisits'] = df['totals.newVisits'].fillna(0)
    
    return df

In [20]:
%%time

train = feature_totals(train)

Wall time: 2 ms


In [21]:
print(train.shape)
train.head()

(1000, 33)


,channelGrouping,fullVisitorId,visitNumber,device.deviceCategory,device.operatingSystem,geoNetwork.continent,geoNetwork.country,geoNetwork.region,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.source,contentGroup.contentGroup2,contentGroup.contentGroup3,eventInfo.eventAction,Year,Month,Day,is_holiday,device_browser,is_inbasket
0,Organic Search,3162355547410993243,1,desktop,Windows,Europe,Germany,NaN,1,1,1,1,1,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,water bottle,google,Bags,NaN,NaN,2017,10,16,0,Firefox,0
1,Referral,8934116514970143966,6,desktop,Chrome OS,Americas,United States,California,0,2,0,2,2,28,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,sites.google.com,NaN,NaN,NaN,2017,10,16,0,Chrome,0
2,Direct,7992466427990357681,1,mobile,Android,Americas,United States,NaN,0,2,1,2,1,38,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,(direct),NaN,NaN,NaN,2017,10,16,0,Chrome,0
3,Organic Search,9075655783635761930,1,desktop,Windows,Asia,Turkey,NaN,0,2,1,2,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,google,NaN,NaN,NaN,2017,10,16,0,Chrome,0
4,Organic Search,6960673291025684308,1,desktop,Windows,Americas,Mexico,NaN,0,2,1,2,1,52,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,google,NaN,NaN,NaN,2017,10,16,0,Chrome,0


In [22]:
def feature_discountad(df):
    discountad = []
    for each in df['trafficSource.adContent']:
        each = str(each)
        if 'discount' in each:
            discountad.append(1)
        else:
            discountad.append(0)
    
    is_discountad = pd.DataFrame(discountad).rename(columns = {0:'is_discountad'})
    
    df = pd.concat([df, is_discountad], axis = 1)
    
    return df

In [23]:
%%time

train = feature_discountad(train)

Wall time: 5.99 ms


In [24]:
print(train.shape)
train.head()

(1000, 34)


,channelGrouping,fullVisitorId,visitNumber,device.deviceCategory,device.operatingSystem,geoNetwork.continent,geoNetwork.country,geoNetwork.region,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.source,contentGroup.contentGroup2,contentGroup.contentGroup3,eventInfo.eventAction,Year,Month,Day,is_holiday,device_browser,is_inbasket,is_discountad
0,Organic Search,3162355547410993243,1,desktop,Windows,Europe,Germany,NaN,1,1,1,1,1,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,water bottle,google,Bags,NaN,NaN,2017,10,16,0,Firefox,0,0
1,Referral,8934116514970143966,6,desktop,Chrome OS,Americas,United States,California,0,2,0,2,2,28,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,sites.google.com,NaN,NaN,NaN,2017,10,16,0,Chrome,0,0
2,Direct,7992466427990357681,1,mobile,Android,Americas,United States,NaN,0,2,1,2,1,38,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,(direct),NaN,NaN,NaN,2017,10,16,0,Chrome,0,0
3,Organic Search,9075655783635761930,1,desktop,Windows,Asia,Turkey,NaN,0,2,1,2,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,google,NaN,NaN,NaN,2017,10,16,0,Chrome,0,0
4,Organic Search,6960673291025684308,1,desktop,Windows,Americas,Mexico,NaN,0,2,1,2,1,52,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,google,NaN,NaN,NaN,2017,10,16,0,Chrome,0,0


In [25]:
# for column in train.columns:
#     print(column)
#     print(train[column].unique())
#     print('\n')

In [26]:
def feature_encode(df):
    categorical = ['channelGrouping',
                 'device.deviceCategory',
                 'device.operatingSystem',
                 'geoNetwork.continent',
                 'geoNetwork.country',
                 'geoNetwork.region',
                 'trafficSource.adContent',
                 'trafficSource.adwordsClickInfo.adNetworkType',
                 'trafficSource.adwordsClickInfo.slot',
                 'trafficSource.campaign',
                 'trafficSource.keyword',
                 'trafficSource.source',
                 'contentGroup.contentGroup2',
                 'contentGroup.contentGroup3',
                 'eventInfo.eventAction',
                 'Year',
                 'Month',
                 'Day',
                 'is_holiday',
                 'device_browser',
                 'is_inbasket',
                 'is_discountad']
    numeric = ['visitNumber',
             'totals.bounces',
             'totals.hits',
             'totals.newVisits',
             'totals.pageviews',
             'totals.sessionQualityDim',
             'totals.timeOnSite',
             'totals.transactions',
             'trafficSource.adwordsClickInfo.page']
    
    floatcol = ['totals.totalTransactionRevenue',
             'totals.transactionRevenue']
    
    for each in categorical:
        df[each] = df[each].astype('category')
    
    for each in numeric:
        df[each] = df[each].astype('float32')
    
    for each in floatcol:
        df[each] = df[each].astype('float')
    
    return df

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 34 columns):
channelGrouping                                 1000 non-null object
fullVisitorId                                   1000 non-null object
visitNumber                                     1000 non-null int64
device.deviceCategory                           1000 non-null object
device.operatingSystem                          998 non-null object
geoNetwork.continent                            999 non-null object
geoNetwork.country                              999 non-null object
geoNetwork.region                               451 non-null object
totals.bounces                                  1000 non-null object
totals.hits                                     1000 non-null object
totals.newVisits                                1000 non-null object
totals.pageviews                                1000 non-null object
totals.sessionQualityDim                        1000 non-null object
to

In [28]:
%%time

train = feature_encode(train)

Wall time: 39.9 ms


In [29]:
print(train.shape)
train.head()

(1000, 34)


,channelGrouping,fullVisitorId,visitNumber,device.deviceCategory,device.operatingSystem,geoNetwork.continent,geoNetwork.country,geoNetwork.region,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.totalTransactionRevenue,totals.transactionRevenue,totals.transactions,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.source,contentGroup.contentGroup2,contentGroup.contentGroup3,eventInfo.eventAction,Year,Month,Day,is_holiday,device_browser,is_inbasket,is_discountad
0,Organic Search,3162355547410993243,1.0,desktop,Windows,Europe,Germany,NaN,1.0,1.0,1.0,1.0,1.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,water bottle,google,Bags,NaN,NaN,2017,10,16,0,Firefox,0,0
1,Referral,8934116514970143966,6.0,desktop,Chrome OS,Americas,United States,California,0.0,2.0,0.0,2.0,2.0,28.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,sites.google.com,NaN,NaN,NaN,2017,10,16,0,Chrome,0,0
2,Direct,7992466427990357681,1.0,mobile,Android,Americas,United States,NaN,0.0,2.0,1.0,2.0,1.0,38.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,(direct),NaN,NaN,NaN,2017,10,16,0,Chrome,0,0
3,Organic Search,9075655783635761930,1.0,desktop,Windows,Asia,Turkey,NaN,0.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,google,NaN,NaN,NaN,2017,10,16,0,Chrome,0,0
4,Organic Search,6960673291025684308,1.0,desktop,Windows,Americas,Mexico,NaN,0.0,2.0,1.0,2.0,1.0,52.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,google,NaN,NaN,NaN,2017,10,16,0,Chrome,0,0


In [31]:
train_id = train["fullVisitorId"].values

In [30]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 34 columns):
channelGrouping                                 1000 non-null category
fullVisitorId                                   1000 non-null object
visitNumber                                     1000 non-null float32
device.deviceCategory                           1000 non-null category
device.operatingSystem                          998 non-null category
geoNetwork.continent                            999 non-null category
geoNetwork.country                              999 non-null category
geoNetwork.region                               451 non-null category
totals.bounces                                  1000 non-null float32
totals.hits                                     1000 non-null float32
totals.newVisits                                1000 non-null float32
totals.pageviews                                1000 non-null float32
totals.sessionQualityDim                        1000 